In [1]:
from collections import defaultdict
import os
import re
import sys

REPO_ROOT = os.environ["REPO_PATH"]
PROJECT_NAME = "advent-of-code-2023"
sys.path.append(os.path.join(REPO_ROOT, PROJECT_NAME))

import utils

# Common functions

In [2]:
def parse_maps(mapping_list: list[str]) -> list:
    mapping = defaultdict(list)
    key = ""
    values = []
    for line in mapping_list:
        if re.match(r"\d+", line):
            line = [int(x) for x in line.split(" ")]
            mapping[key].append(line)
        else:
            key = line.split(" map:")[0]
    return dict(mapping)


def find_mapped_value(mapping: list[list[int]], value: int) -> int:
    for l in mapping:
        lower_key = l[1]
        higher_key = l[1] + l[2]
        delta = l[0] - l[1]
        if value >= lower_key and value < higher_key:
            return value + delta
    return value


def find_location(seed: int, mapping: dict) -> int:
    maps = mapping.copy()
    if len(maps) == 0:
        return seed
    keys = [k for k in maps.keys()]
    mapping_list = maps.pop(keys[0])
    mapped_value = find_mapped_value(mapping_list, seed)
    return find_location(mapped_value, maps)

# Extract data

In [3]:
puzzle_data = utils.load_data(test=False)
puzzle_data = [x for x in puzzle_data if x != ""]

In [4]:
seeds = puzzle_data[0].split(": ")[1].split(" ")
seeds = [int(x) for x in seeds]

In [5]:
maps = puzzle_data[1:]
maps = parse_maps(maps)

# Puzzle 1

In [6]:
locations = []
for seed in seeds:
    locations.append(find_location(seed, maps))

In [7]:
print(f"{min(locations)=}")

min(locations)=551761867


# Puzzle 2

In [8]:
class ValueNotInBounds(Exception):
    pass


def check_in_ranges(value: int, ranges: list[tuple[int, int]]) -> bool:
    for r in ranges:
        if value >= r[0] and value <= r[1]:
            return value
    raise ValueNotInBounds

In [9]:
seeds_ranges = []
for s in range(0, len(seeds), 2):
    seeds_ranges.append((seeds[s], seeds[s] + seeds[s + 1]))

In [10]:
all_origins_lower_bounds = []
for map_ in maps.values():
    for m in map_:
        try:
            lower_bound = check_in_ranges(m[1], seeds_ranges)
        except ValueNotInBounds:
            continue
        else:
            all_origins_lower_bounds.append(lower_bound)

In [11]:
locations = []
for seed in all_origins_lower_bounds:
    locations.append(find_location(seed, maps))

In [12]:
print(f"{min(locations)=}")

min(locations)=57451709
